In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit

Using matplotlib backend: Qt5Agg


In [3]:
from Experimentos2 import Experimentos, labelLines

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - frente.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - frente.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\

In [4]:
#CORRER lectura de archivos:
velos = pd.read_excel('Velocidades y punto.xlsx')
PropTerm = pd.read_excel('PropiedadesTermicas.xlsx')

#datos para cálculos
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2

#propiedades medidas a temperatura ambiente
Conduct = PropTerm['Conductividad'].mean() #conductividad
Cap = PropTerm['Capacidad'].mean()*10**6 #calor especifico volumetrico
Difus = PropTerm['Difusividad'].mean()*10**(-7) #difusividad
Resist = PropTerm['Resistividad'].mean() #resistividad termica
Cp = Cap/1340 #calor especifico masico

#defino funciones de reologia
def YieldStg(temp):
    return 6220*temp**(-1.36)
def Konsist(temp):
    return 1*10**6 * temp**(-2.61)
def FlowInd(temp):
    return 0.0046*temp + 0.145

#actualizo y guardo datos a usar mas tarde
for exp in Experimentos:
    #updateo densidad
    exp.param['densidad']=1340
    #calculo cos angulo
    exp.param['slope sen'] = np.sin(np.radians(exp.param['angulo inclinacion']))
    exp.param['slope cos'] = np.cos(np.radians(exp.param['angulo inclinacion']))
    
#Añado velos a params x facilidad
for exp in Experimentos:
    for i in range(33):
        if exp.param['NOMBRE'][0]==velos.iloc[i,:]['nombre']:
            exp.param['v1'] = velos.iloc[i,:]['v1']
            exp.param['v2'] = velos.iloc[i,:]['v2']
            exp.param['L*'] = velos.iloc[i,:]['L*']
            exp.param['t*'] = velos.iloc[i,:]['t*']

In [14]:
#calculos varios

for exp in Experimentos:
    exp.param['tiempo fin eyeccion'] = exp.param['volumen total teorico']/exp.param['caudal']

#Estimo Jeffreys parte primera
for exp in Experimentos:
    if exp.param['NOMBRE'][0] == '1cc48C':
        continue
    exp.param['numerador'] = 9.8*exp.param['densidad'][0]*exp.param['slope sen'][0]*(exp.perfil['perfil: espesor'].max())**2
    if exp.param['v1'][0] > 0:
        exp.param['denominador1'] = 3*exp.param['v1'][0]
    if exp.param['v2'][0] > 0:
        exp.param['denominador2'] = 3*exp.param['v2'][0]
        
#Estimo Jeffreys parte segunda y final
for exp in Experimentos:
    if exp.param['NOMBRE'][0] == '1cc48C':
        continue
    if exp.param['v1'][0] > 0:
        exp.param['visc1'] = exp.param['numerador'][0]/exp.param['denominador1'][0]
    if exp.param['v2'][0] > 0:
        exp.param['visc2'] = exp.param['numerador'][0]/exp.param['denominador2'][0]
        #print(exp,exp.param['visc2'])
        
#Largo según Lister para flujos newtonianos
for exp in Experimentos:
    exp.dinamica['numerador'] = exp.param['caudal'][0]*exp.param['caudal'][0]*exp.param['densidad'][0]*9.8*exp.param['slope sen'][0]
    if exp.param['v1'][0] > 0:
        exp.dinamica['denominador'] = exp.param['visc1'][0]*exp.dinamica['avance: ancho maximo']*exp.dinamica['avance: ancho maximo']
        exp.dinamica['division'] = np.cbrt(exp.dinamica['numerador'][0]/exp.dinamica['denominador'])
        exp.dinamica['Lteonew'] = exp.dinamica['division']*exp.dinamica['avance: tiempo']
        
#Calculo y Guardo el Bingham para todos los exp
for exp in Experimentos:
    temp = exp.param['t max'][0]
    exp.param['Ty'] = YieldStg(temp)
    exp.param['K'] = Konsist(temp)
    exp.param['FlowI'] = FlowInd(temp)
    exp.param['H'] = exp.perfil['perfil: espesor'].max()
    exp.param['Lmax'] = exp.dinamica['avance: distancia desde punto eyeccion'].max()
    exp.param['e'] = exp.param['H'][0]/exp.param['Lmax'][0]
    #Bingham B=tp/e*rho*g*H*cos(o) ec (2.6) shallow viscoplastic flow on an inclined plane de balmforth
    exp.param['Bingham'] = exp.param['Ty'][0] / (exp.param['e'][0]*exp.param['densidad'][0]*9.8*exp.param['H'][0]*exp.param['slope cos'][0])
    #B2 y B3 bingham 2 ec 3.12 cooling domes of viscoplastic fluids
    if exp.param['v1'][0] > 0:
        exp.param['B2'] = exp.param['Ty'][0]*exp.param['H'][0]/(exp.param['visc1'][0]*exp.param['v1'][0])
    exp.param['B3'] = exp.param['Ty'][0]*exp.param['Lmax'][0]/(exp.param['densidad'][0]*9.8*exp.param['H'][0]**2)

#Calculo Reynolds
for exp in Experimentos:
    if exp.param['v1'][0] > 0:
        exp.param['Re'] = exp.param['densidad'][0]*exp.param['v1'][0]*exp.param['L*'][0]/exp.param['visc1'][0]
        #print(exp,exp.param['Re'])

In [28]:
#FUNCION Q DEVUELVE RANGO [1-7] SEGUN TEMP MAXIMA DEL EXP
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA

import matplotlib.patches as mpatches

colors = ['burlywood','sandybrown','salmon','red','crimson','brown','maroon']

pat1 = mpatches.Patch(color=colors[0], label='temp < 40')
pat2 = mpatches.Patch(color=colors[1], label='temp < 45')
pat3 = mpatches.Patch(color=colors[2], label='temp < 50')
pat4 = mpatches.Patch(color=colors[3], label='temp < 55')
pat5 = mpatches.Patch(color=colors[4], label='temp < 60')
pat6 = mpatches.Patch(color=colors[5], label='temp < 65')
pat7 = mpatches.Patch(color=colors[6], label='temp < 70')

In [12]:
#se arman GRUPOS SEGUN CAUDAL
listadividircaudales = []
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    listadividircaudales.append(holder)
        
from itertools import groupby
count = 0
for key, group in groupby(sorted(listadividircaudales, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [46]:
listaplotsperfildinamica = []
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['v1'] = exp.param['v1'][0]    
    holder['v2'] = exp.param['v2'][0]
    holder['t*'] = exp.param['t*'][0]
    holder['L*'] = exp.param['L*'][0]
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.perfil['perfil: espesor']#toda la columna
    holder['largoperfil'] = exp.perfil['perfil: distancia']#toda la columna
    holder['ancho'] = exp.param['result: ancho max final']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    #holder['uso'] = exp.param['uso manjar']
    holder['teyec'] = exp.param['tiempo fin eyeccion']
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']#toda la columna
    holder['ancho'] = exp.dinamica['avance: ancho maximo']#toda la columna
    holder['Bingham'] = exp.param['Bingham'][0]
    holder['yDataxmin'] = exp.dinamica['avance: distancia hacia atrás']    
    holder['rango temperatura'] = rango_temperatura(exp.param["t max"][0])


    listaplotsperfildinamica.append(holder)

In [39]:
for exp in listaplotsperfildinamica:
    print(exp['rango temperatura'],exp['nombre'])

4 5cc50C
4 1cc54C
4 25cc54C
6 10cc60C
6 25cc60C
5 1cc59C
3 1cc47C
4 1cc55C
5 25cc58C
7 25cc70C
7 10cc67C
5 10cc59C
6 5cc63C
5 5cc59C
3 10cc49C
6 25cc62C
6 1cc62C
4 5cc53C
5 5a25cc56C
5 5cc57C
4 25cc50C
5 10cc55C
1 1cc24C
5 10cc57C
1 10cc24C
5 5cc55C
5 5cc58C
3 1cc48C
4 1cc51C
3 5cc48C
4 10cc50C
4 1cc54,7C
4 1cc52C


In [41]:
#LISTOOOO
#5 GRAFICOS DE PERFIL AGRUPADOS POR CAUDAL Y COLOREADOS SEGUN TEMEPRATURA 
#SOLO PRIMER USO DEL MANJAR
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaplotsperfildinamica, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #if exp['uso'] == 1: 
        if exp['angulo'] == 15:
            ax.plot(exp['largoperfil'], exp['espesor'], color=colors[exp['rango temperatura']-1],
                    linestyle= '--')
        else:
            try:
                ax.plot(exp['largoperfil'], exp['espesor'], color=colors[exp['rango temperatura']-1])
            except Exception as e:
                print(e, exp['nombre'])
        
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05
